# Import Dataset

In [90]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

from Preprocessing_functions import *

In [91]:
train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')
test_data = pd.read_csv('test_data.csv', index_col='Claim Identifier')

/var/folders/z6/_vn8cndd4x5dmx2cpjfdkjdr0000gn/T/ipykernel_20816/3470921380.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', index_col='Claim Identifier')


In [92]:
train_data.isna().sum()

Accident Date                          23134
Age at Injury                          19445
Alternative Dispute Resolution         19445
Assembly Date                              0
Attorney/Representative                19445
Average Weekly Wage                    48096
Birth Year                             48523
C-2 Date                               34005
C-3 Date                              406226
Carrier Name                           19445
Carrier Type                           19445
Claim Injury Type                      19445
County of Injury                       19445
COVID-19 Indicator                     19445
District Name                          19445
First Hearing Date                    442673
Gender                                 19445
IME-4 Count                           460668
Industry Code                          29403
Industry Code Description              29403
Medical Fee Region                     19445
OIICS Nature of Injury Description    593471
WCIO Cause

# Preprocessing

In [93]:
CODE_COLUMNS = ['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

DESCRIPTION_COLUMNS = ['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description']

BOOLEAN_COLUMNS = ['Alternative Dispute Resolution', 'Attorney/Representative','COVID-19 Indicator']

date_order = ['Accident Date', 'C-2 Date','C-3 Date','Assembly Date', 'First Hearing Date']

In [94]:
train_data = train_data[~(train_data.drop(columns=['Assembly Date']).isna().all(axis=1) & train_data['Assembly Date'].notna())]

In [95]:
X = train_data.drop(columns=['Claim Injury Type', 'WCB Decision', 'Agreement Reached'])
y = train_data['Claim Injury Type']

In [96]:
y.isna().sum()

np.int64(0)

In [97]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [99]:
def drop_description_columns(X_train, X_val):
    """
    Drop all columns in X_train and X_val that contain the word 'description' in their names (case-insensitive).
    """
    # Identify columns containing the word 'description'
    description_columns = X_train.columns[X_train.columns.str.contains('description', case=False, na=False)]
    
    # Drop these columns from both DataFrames
    X_train = X_train.drop(description_columns, axis=1)
    X_val = X_val.drop(description_columns, axis=1)
    
    return X_train, X_val

# Call the function
X_train, X_val = drop_description_columns(X_train, X_val)

In [ ]:
scaling_minmax_col = ['Age at Injury', 'Birth Year', 'IME-4 Count', 'Number of Dependents']
scaling_standart_col = X_train.columns[X_train.columns.str.contains('Date')].drop(['Has First Hearing Date', 'Has C-2 Date', 'Has C-3 Date'])


In [ ]:


def preprocessing_complex(X_train, X_val):
    convert_to_timestamp(X_train, X_val, date_order)
    convert_to_bool(X_train, X_val, col_names=BOOLEAN_COLUMNS)
    impute_mean_numerical(X_train, X_val, ['C-2 Date'])
    fillna_zip_code(X_train, X_val)
    fillnan_accident_date(X_train,X_val)
    fillnan_birth_year(X_train,X_val)
    feature_creation_has_Cdate (X_train, X_val)
    fill_missing_with_mode(X_train, X_val)
    drop_description_columns(X_train, X_val)
    scaling_standard(X_train, X_val, columns)
    encoding_onehot(X_train, X_val, columns)
    return X_train, X_val
preprocessing_complex(X_train, X_val)

X_train.info()






/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:296: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
/Users/philippedutranoit/document ordi/nova/machine learning /project/ML_Group36/src/Preprocessing_functions.py:297: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work 

<class 'pandas.core.frame.DataFrame'>
Index: 459220 entries, 5785935 to 6027959
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Accident Date                   459220 non-null  float64
 1   Age at Injury                   459220 non-null  float64
 2   Alternative Dispute Resolution  459216 non-null  object 
 3   Assembly Date                   459220 non-null  float64
 4   Attorney/Representative         459220 non-null  object 
 5   Average Weekly Wage             436244 non-null  float64
 6   Birth Year                      459220 non-null  float64
 7   C-2 Date                        459220 non-null  float64
 8   C-3 Date                        149655 non-null  float64
 9   Carrier Name                    459220 non-null  object 
 10  Carrier Type                    459220 non-null  object 
 11  County of Injury                459220 non-null  object 
 12  COVID-19 Indic

In [ ]:
X_train.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents,time_diff C2 accident
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5785935,1.628122e+09,49.0,False,1.628554e+09,True,744.06,3.153600e+07,1.628554e+09,1.630282e+09,PROPERTY AND CASUALTY,...,NaN,29.0,ON SAME LEVEL,59.0,"ALL OTHER SPECIFIC INJURIES, NOC",42.0,LOWER BACK AREA,11432,6.0,-432000.0
6090033,1.663114e+09,56.0,False,1.663718e+09,True,845.29,-1.262304e+08,1.663718e+09,1.664237e+09,EVEREST PREMIER INSURANCE,...,NaN,45.0,COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE,52.0,STRAIN OR TEAR,25.0,SOFT TISSUE,11367,5.0,-604800.0
6136197,1.667779e+09,48.0,False,1.668730e+09,False,0.00,1.262304e+08,1.668730e+09,NaN,CHARTER OAK FIRE INS CO,...,NaN,68.0,STATIONARY OBJECT,10.0,CONTUSION,12.0,BRAIN,10956,2.0,-950400.0
6019545,1.599005e+09,55.0,False,1.655770e+09,False,0.00,-1.735690e+09,1.655770e+09,1.647389e+09,NEW HAMPSHIRE INSURANCE CO,...,NaN,60.0,"STRAIN OR INJURY BY, NOC",78.0,CARPAL TUNNEL SYNDROME,34.0,WRIST,08904,2.0,-56764800.0
5792247,1.628813e+09,53.0,False,1.629245e+09,True,668.08,-6.315840e+07,1.629245e+09,1.638230e+09,STATE INSURANCE FUND,...,NaN,29.0,ON SAME LEVEL,10.0,CONTUSION,18.0,SOFT TISSUE,12158,6.0,-432000.0


# Model Training

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


# Range of k values to test
k_range = range(1, 10)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store mean accuracy for each k
mean_accuracies = []

# Perform K-Fold for each k
for k in k_range:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets for this fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the KNN model
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        
        # Evaluate on the test set
        y_pred = knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    # Compute the mean accuracy for this k
    mean_accuracies.append(np.mean(accuracies))

# Find the optimal k
optimal_k = k_range[np.argmax(mean_accuracies)]
print(f"The optimal number of neighbors is {optimal_k}.")

# Plot the results
import matplotlib.pyplot as plt
plt.plot(k_range, mean_accuracies)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Mean Accuracy')
plt.title('Optimal k Selection using K-Fold Cross-Validation')
plt.show()

# Final training with optimal k
final_knn = KNeighborsClassifier(n_neighbors=optimal_k)
final_knn.fit(X, y)
print(f"Model trained with optimal k={optimal_k}.")

  